Get all datasets and the according attribute values which contain (at least) one link without _https_ which also works with the replaced protocol.

In [ ]:
! pip install urlextract

In [2]:
import pandas as pd # 0.23.0
import numpy as np
import requests as r
import time
import urllib3

In [3]:
df = pd.read_csv('INT_linksToChange.csv')
df

,title,type,attribute,id,old_link,new_link
0,Aktuelle Pilot- und Demonstrationsprojekte des...,resources,download_url,ff6a28fe-f691-4ee1-87df-50ef4d8decf6,https://data.geo.admin.ch/ch.bfe.energieforsch...,-
1,Aktuelle Pilot- und Demonstrationsprojekte des...,resources,url,ff6a28fe-f691-4ee1-87df-50ef4d8decf6,https://data.geo.admin.ch/ch.bfe.energieforsch...,-
2,Analyse des schweizerischen Energieverbrauchs ...,dataset,url,cd67848c-9051-44e1-9e72-11c153b46abb,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3
3,Analyse des schweizerischen Energieverbrauchs ...,resources,url,d904c440-fdb6-4e98-baff-52f73b1f6a06,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3
4,Angebote der Innovationsförderung im Energiebe...,dataset,url,44f9dd93-6bd0-4d56-8650-e472b44b72b0,http://www.bfe.admin.ch/ogd4,https://www.bfe.admin.ch/ogd4
...,...,...,...,...,...,...
122,Windatlas Schweiz: Jahresmittel der modelliert...,resources,download_url,98f52d31-ed69-4634-b95d-2ba786f83cfb,https://data.geo.admin.ch/ch.bfe.windenergie-g...,-
123,Windatlas Schweiz: Jahresmittel der modelliert...,resources,url,98f52d31-ed69-4634-b95d-2ba786f83cfb,https://data.geo.admin.ch/ch.bfe.windenergie-g...,-
124,Wochenstatistik Elektrizitätsbilanz - Erzeugun...,dataset,url,979f37ed-fcb5-4400-802b-be0bb2833ea4,http://www.bfe.admin.ch/ogd51,https://www.bfe.admin.ch/ogd51
125,Wochenstatistik Elektrizitätsbilanz - Erzeugun...,resources,uri,2d524296-9638-4ae5-b623-07b253540f7c,https://ckan.opendata.swiss/dataset/9ab8ccdd-a...,-


In [5]:
tmp = df[df.attribute != 'relations'].copy()
tmp

,title,type,attribute,id,old_link,new_link
0,Aktuelle Pilot- und Demonstrationsprojekte des...,resources,download_url,ff6a28fe-f691-4ee1-87df-50ef4d8decf6,https://data.geo.admin.ch/ch.bfe.energieforsch...,-
1,Aktuelle Pilot- und Demonstrationsprojekte des...,resources,url,ff6a28fe-f691-4ee1-87df-50ef4d8decf6,https://data.geo.admin.ch/ch.bfe.energieforsch...,-
2,Analyse des schweizerischen Energieverbrauchs ...,dataset,url,cd67848c-9051-44e1-9e72-11c153b46abb,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3
3,Analyse des schweizerischen Energieverbrauchs ...,resources,url,d904c440-fdb6-4e98-baff-52f73b1f6a06,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3
4,Angebote der Innovationsförderung im Energiebe...,dataset,url,44f9dd93-6bd0-4d56-8650-e472b44b72b0,http://www.bfe.admin.ch/ogd4,https://www.bfe.admin.ch/ogd4
...,...,...,...,...,...,...
122,Windatlas Schweiz: Jahresmittel der modelliert...,resources,download_url,98f52d31-ed69-4634-b95d-2ba786f83cfb,https://data.geo.admin.ch/ch.bfe.windenergie-g...,-
123,Windatlas Schweiz: Jahresmittel der modelliert...,resources,url,98f52d31-ed69-4634-b95d-2ba786f83cfb,https://data.geo.admin.ch/ch.bfe.windenergie-g...,-
124,Wochenstatistik Elektrizitätsbilanz - Erzeugun...,dataset,url,979f37ed-fcb5-4400-802b-be0bb2833ea4,http://www.bfe.admin.ch/ogd51,https://www.bfe.admin.ch/ogd51
125,Wochenstatistik Elektrizitätsbilanz - Erzeugun...,resources,uri,2d524296-9638-4ae5-b623-07b253540f7c,https://ckan.opendata.swiss/dataset/9ab8ccdd-a...,-


In [7]:
t = tmp[tmp.new_link != '-'].copy()
t

,title,type,attribute,id,old_link,new_link
2,Analyse des schweizerischen Energieverbrauchs ...,dataset,url,cd67848c-9051-44e1-9e72-11c153b46abb,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3
3,Analyse des schweizerischen Energieverbrauchs ...,resources,url,d904c440-fdb6-4e98-baff-52f73b1f6a06,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3
4,Angebote der Innovationsförderung im Energiebe...,dataset,url,44f9dd93-6bd0-4d56-8650-e472b44b72b0,http://www.bfe.admin.ch/ogd4,https://www.bfe.admin.ch/ogd4
5,Angebote der Innovationsförderung im Energiebe...,resources,url,7984270f-76e8-472d-879f-f1b8557a6dde,http://www.bfe.admin.ch/ogd4,https://www.bfe.admin.ch/ogd4
6,Beteiligung des BFE an internationalen Energie...,dataset,url,02169deb-6abb-40e1-9d1b-429cfcdc66ef,http://www.bfe.admin.ch/ogd5,https://www.bfe.admin.ch/ogd5
...,...,...,...,...,...,...
108,Thermische Stromproduktion inklusive Wärmekraf...,resources,url,56e570c3-6769-4bd1-97c4-056a5fdcb116,http://www.bfe.admin.ch/ogd45,https://www.bfe.admin.ch/ogd45
109,Verkaufszahlenbasierte Energieeffizienzanalyse...,dataset,url,eb08e2e0-49c0-44cb-b7d3-a60495af7c3d,http://www.bfe.admin.ch/ogd46,https://www.bfe.admin.ch/ogd46
110,Verkaufszahlenbasierte Energieeffizienzanalyse...,resources,url,744748d5-1778-48a3-acf7-ca60a90a38b4,http://www.bfe.admin.ch/ogd46,https://www.bfe.admin.ch/ogd46
124,Wochenstatistik Elektrizitätsbilanz - Erzeugun...,dataset,url,979f37ed-fcb5-4400-802b-be0bb2833ea4,http://www.bfe.admin.ch/ogd51,https://www.bfe.admin.ch/ogd51


In [12]:
t.iloc[0]['title']

'Analyse des schweizerischen Energieverbrauchs nach Verwendungszwecken'

In [8]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

s = r.Session()

proxies = None

private_api_key = 'cc1c806e-1fd9-4180-a6bf-7a5304ca503a'

In [9]:
patch_url = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_patch'

In [10]:
header = {'Authorization': private_api_key}

In [17]:
b = t[t.title == 'Analyse des schweizerischen Energieverbrauchs nach Verwendungszwecken'].copy()
b

,title,type,attribute,id,old_link,new_link
2,Analyse des schweizerischen Energieverbrauchs ...,dataset,url,cd67848c-9051-44e1-9e72-11c153b46abb,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3
3,Analyse des schweizerischen Energieverbrauchs ...,resources,url,d904c440-fdb6-4e98-baff-52f73b1f6a06,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3


In [20]:
for index, row in b.iterrows():

  patch_url = ''

  if row['type'] == 'resources':
    patch_url = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/resource_patch'
  else:
    patch_url = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_patch'

  
  data = {'id': str(row['id']),
         row['attribute'] : str(row['new_link'])}

  print(data)
    
  x = s.post(patch_url,data=data,proxies=proxies,headers=header,verify=False)
  print(x.status_code)
  print()

{'id': 'cd67848c-9051-44e1-9e72-11c153b46abb', 'url': 'https://www.bfe.admin.ch/ogd3'}
200

{'id': 'd904c440-fdb6-4e98-baff-52f73b1f6a06', 'url': 'https://www.bfe.admin.ch/ogd3'}
200



In [ ]:
for i in data = {'id': str(tmp['id'].values[0]),
        tmp['attribute'].values[0] : str(tmp['new_link'].values[0])}
    
print(data)
    
x = s.post(patch_url,data=data,proxies=proxies,headers=header,verify=False)
print(x.status_code)

{'id': '979f37ed-fcb5-4400-802b-be0bb2833ea4', 'url': 'https://www.bfe.admin.ch/ogd51'}
200


In [ ]:
tmp['attribute'].values[0]

'url'

In [ ]:
tmp['id'].values[0]

'979f37ed-fcb5-4400-802b-be0bb2833ea4'

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
patch_url = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/resource_patch'

In [ ]:
test2

,title,type,attribute,id,old_link,new_link
103,"Thermische Netze (Nahwärme, Fernwärme, Fernkälte)",resources,download_url,be78136d-fc98-4447-8a10-96cce2c3f260,http://data.geo.admin.ch/ch.bfe.thermische-net...,https://data.geo.admin.ch/ch.bfe.thermische-ne...
104,"Thermische Netze (Nahwärme, Fernwärme, Fernkälte)",resources,download_url,2b0c3950-a415-43be-a18d-680f92dc71d2,http://data.geo.admin.ch/ch.bfe.thermische-net...,https://data.geo.admin.ch/ch.bfe.thermische-ne...
105,"Thermische Netze (Nahwärme, Fernwärme, Fernkälte)",resources,url,be78136d-fc98-4447-8a10-96cce2c3f260,http://data.geo.admin.ch/ch.bfe.thermische-net...,https://data.geo.admin.ch/ch.bfe.thermische-ne...
106,"Thermische Netze (Nahwärme, Fernwärme, Fernkälte)",resources,url,2b0c3950-a415-43be-a18d-680f92dc71d2,http://data.geo.admin.ch/ch.bfe.thermische-net...,https://data.geo.admin.ch/ch.bfe.thermische-ne...


In [ ]:
frameList = []

for i in test2['id'].unique():
  frameList.append(test2[test2.id == i])

In [ ]:
frameList[0]

,title,type,attribute,id,old_link,new_link
103,"Thermische Netze (Nahwärme, Fernwärme, Fernkälte)",resources,download_url,be78136d-fc98-4447-8a10-96cce2c3f260,http://data.geo.admin.ch/ch.bfe.thermische-net...,https://data.geo.admin.ch/ch.bfe.thermische-ne...
105,"Thermische Netze (Nahwärme, Fernwärme, Fernkälte)",resources,url,be78136d-fc98-4447-8a10-96cce2c3f260,http://data.geo.admin.ch/ch.bfe.thermische-net...,https://data.geo.admin.ch/ch.bfe.thermische-ne...


In [ ]:
for index, row in frameList[1].iterrows():
  
  data = {'id': str(row['id']),
         row['attribute'] : str(row['new_link'])}

  print(data)
    
  x = s.post(patch_url,data=data,proxies=proxies,headers=header,verify=False)
  print(x.status_code)
  print()

{'id': '2b0c3950-a415-43be-a18d-680f92dc71d2', 'download_url': 'https://data.geo.admin.ch/ch.bfe.thermische-netze/xtf/2056/ch.bfe.thermische-netze.zip'}
200

{'id': '2b0c3950-a415-43be-a18d-680f92dc71d2', 'url': 'https://data.geo.admin.ch/ch.bfe.thermische-netze/xtf/2056/ch.bfe.thermische-netze.zip'}
500

